# Eternal Return VLM Fine-tuning

이터널리턴 전문가 VLM (데비&마를렌 봇) 파인튜닝 노트북

## 목표
- 이터널리턴 아이템/캐릭터/특성 이미지 인식
- Tool Use 패턴 학습
- 데비&마를렌 페르소나 적용

## 모델
- **SmolVLM2-2.2B** (Colab 무료 T4에서 가능)
- 또는 **Qwen2.5-VL-3B** (더 좋은 성능)

## 1. 환경 설정

In [ ]:
# 필수 패키지 설치
!pip install -q transformers>=4.45.0
!pip install -q trl>=0.12.0
!pip install -q peft>=0.13.0
!pip install -q accelerate>=0.34.0
!pip install -q bitsandbytes>=0.44.0
!pip install -q datasets
!pip install -q pillow
!pip install -q wandb  # 학습 로깅용 (선택)

In [ ]:
# GPU 확인
!nvidia-smi

## 2. 데이터셋 업로드

로컬에서 생성한 데이터셋과 이미지를 업로드해야 합니다.

### 필요한 파일:
1. `eternal_return_vlm_dataset.json` - 학습 데이터
2. `emojis/` 폴더 - 이미지들 (zip으로 압축해서 업로드)

In [ ]:
# Google Drive 마운트 (데이터 업로드용)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 데이터셋 경로 설정
# Google Drive에 업로드한 경우:
DATASET_PATH = "/content/drive/MyDrive/eternal_return_vlm/eternal_return_vlm_dataset.json"
IMAGES_DIR = "/content/drive/MyDrive/eternal_return_vlm/emojis"

# 또는 직접 업로드한 경우:
# from google.colab import files
# uploaded = files.upload()  # dataset.json 업로드
# DATASET_PATH = "eternal_return_vlm_dataset.json"

import os
print(f"Dataset exists: {os.path.exists(DATASET_PATH)}")
print(f"Images dir exists: {os.path.exists(IMAGES_DIR)}")

## 3. 데이터셋 로드 및 전처리

In [ ]:
import json
from PIL import Image
from pathlib import Path

# 데이터셋 로드
with open(DATASET_PATH, 'r', encoding='utf-8') as f:
    raw_data = json.load(f)

print(f"총 데이터: {len(raw_data)}개")
print(f"샘플: {raw_data[0]}")

In [ ]:
from datasets import Dataset
from PIL import Image
import os

def load_image(image_path):
    """이미지 로드 (상대 경로 처리)"""
    if image_path is None:
        return None
    full_path = os.path.join(IMAGES_DIR, "..", image_path)
    if os.path.exists(full_path):
        return Image.open(full_path).convert("RGB")
    return None

def format_conversation(item):
    """대화를 학습 형식으로 변환"""
    conversations = item.get("conversations", [])
    
    # 이미지가 있는 대화와 없는 대화 구분
    image = load_image(item.get("image"))
    
    # 대화를 텍스트로 변환
    text_parts = []
    for conv in conversations:
        role = conv["from"]
        value = conv["value"]
        
        if role == "human":
            text_parts.append(f"User: {value}")
        elif role == "gpt":
            text_parts.append(f"Assistant: {value}")
        elif role == "tool":
            text_parts.append(f"[Tool Result]: {value}")
    
    return {
        "image": image,
        "text": "\n".join(text_parts)
    }

# 데이터 변환
processed_data = [format_conversation(item) for item in raw_data]

# 이미지가 있는 데이터만 필터링 (VLM 학습용)
processed_data = [d for d in processed_data if d["image"] is not None]

print(f"이미지가 있는 데이터: {len(processed_data)}개")

In [ ]:
# Hugging Face Dataset으로 변환
from datasets import Dataset

dataset = Dataset.from_list(processed_data)
print(dataset)

# Train/Test 분할
dataset = dataset.train_test_split(test_size=0.1, seed=42)
print(f"Train: {len(dataset['train'])}, Test: {len(dataset['test'])}")

## 4. 모델 로드 (SmolVLM2 or Qwen2.5-VL)

In [ ]:
import torch
from transformers import (
    AutoProcessor,
    AutoModelForVision2Seq,
    BitsAndBytesConfig
)

# 모델 선택
MODEL_ID = "HuggingFaceTB/SmolVLM-Instruct"  # 가벼운 모델
# MODEL_ID = "Qwen/Qwen2.5-VL-3B-Instruct"  # 더 좋은 성능 (VRAM 더 필요)

# 4bit 양자화 설정 (VRAM 절약)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# 프로세서 로드
processor = AutoProcessor.from_pretrained(MODEL_ID)

# 모델 로드
model = AutoModelForVision2Seq.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

print(f"모델 로드 완료: {MODEL_ID}")
print(f"모델 메모리: {model.get_memory_footprint() / 1e9:.2f} GB")

## 5. LoRA 설정

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# LoRA 설정
lora_config = LoraConfig(
    r=16,  # LoRA rank
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],  # 주요 레이어
    bias="none",
    task_type="CAUSAL_LM",
)

# 모델 준비
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

# 학습 가능한 파라미터 확인
model.print_trainable_parameters()

## 6. 학습 설정 및 실행

In [ ]:
from trl import SFTTrainer, SFTConfig

# 학습 설정
training_args = SFTConfig(
    output_dir="./eternal_return_vlm_lora",
    
    # 배치 설정
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,
    
    # 학습 설정
    num_train_epochs=3,
    learning_rate=2e-4,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    
    # 최적화
    bf16=True,
    gradient_checkpointing=True,
    optim="adamw_8bit",
    
    # 로깅
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    
    # 기타
    remove_unused_columns=False,
    dataset_text_field="text",
    max_seq_length=512,
)

print("학습 설정 완료")

In [ ]:
# 데이터 콜레이터 (이미지 처리)
def collate_fn(examples):
    texts = [example["text"] for example in examples]
    images = [example["image"] for example in examples]
    
    # 프로세서로 인코딩
    batch = processor(
        text=texts,
        images=images,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512,
    )
    
    # Labels 설정
    batch["labels"] = batch["input_ids"].clone()
    
    return batch

In [ ]:
# Trainer 생성
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=collate_fn,
)

print("Trainer 준비 완료!")

In [ ]:
# 학습 시작!
print("학습 시작...")
trainer.train()
print("학습 완료!")

## 7. 모델 저장

In [ ]:
# LoRA 어댑터 저장
SAVE_PATH = "/content/drive/MyDrive/eternal_return_vlm/lora_adapter"

trainer.save_model(SAVE_PATH)
processor.save_pretrained(SAVE_PATH)

print(f"모델 저장 완료: {SAVE_PATH}")

## 8. 테스트

In [ ]:
# 테스트 이미지로 추론
from PIL import Image

# 테스트 이미지 로드
test_image_path = os.path.join(IMAGES_DIR, "items_graded/202503.png")  # 성법의
test_image = Image.open(test_image_path).convert("RGB")

# 프롬프트
test_prompt = "User: <image>\n이 아이템 뭐야?\nAssistant:"

# 추론
inputs = processor(text=test_prompt, images=test_image, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=True,
        temperature=0.7,
    )

response = processor.decode(outputs[0], skip_special_tokens=True)
print("=" * 50)
print(response)
print("=" * 50)

## 9. Tool Use 테스트

In [ ]:
# Tool Use 패턴 테스트
test_prompt_tool = "User: <image>\n이 아이템 스탯 알려줘\nAssistant:"

inputs = processor(text=test_prompt_tool, images=test_image, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=True,
        temperature=0.7,
    )

response = processor.decode(outputs[0], skip_special_tokens=True)
print("=" * 50)
print("Tool Use 테스트:")
print(response)
print("=" * 50)

# [TOOL_CALL: get_item_stats(...)] 패턴이 나오면 성공!

## 10. 다음 단계

1. **저장된 LoRA 어댑터 다운로드**
   - Google Drive에서 `lora_adapter/` 폴더 다운로드

2. **Discord 봇에 통합**
   - 로컬 GPU 또는 서버에서 모델 로드
   - Tool Call 감지 시 실제 API 호출

3. **성능 개선**
   - 더 많은 데이터로 재학습
   - Qwen2.5-VL-7B로 업그레이드 (RTX 4090에서)